In [1]:
import cPickle as pkl
import pandas as pd
import numpy as np
import os.path

In [2]:
dirname = '../../results_20180423'

In [3]:
descriptor = '_5_transcripts'

In [4]:
files = !ls {dirname}
files = [os.path.join(dirname, filename) for filename in files]
files

['../../results_20180423/test configuration with 2 genes in 50 transcripts, 100 ribosomes, deltat=0.05s_20180423_1546_100_ribosomes_0200s.p',
 '../../results_20180423/test configuration with 2 genes in 50 transcripts, 10 ribosomes, deltat=0.05s_20180423_1545_10_ribosomes_0200s.p',
 '../../results_20180423/test configuration with 2 genes in 50 transcripts, 1 ribosomes, deltat=0.05s_20180423_1545_1_ribosomes_0200s.p',
 '../../results_20180423/test configuration with 2 genes in 50 transcripts, 200 ribosomes, deltat=0.05s_20180423_1547_200_ribosomes_0200s.p',
 '../../results_20180423/test configuration with 2 genes in 50 transcripts, 20 ribosomes, deltat=0.05s_20180423_1545_20_ribosomes_0200s.p',
 '../../results_20180423/test configuration with 2 genes in 50 transcripts, 2 ribosomes, deltat=0.05s_20180423_1545_2_ribosomes_0200s.p',
 '../../results_20180423/test configuration with 2 genes in 50 transcripts, 30 ribosomes, deltat=0.05s_20180423_1545_30_ribosomes_0200s.p',
 '../../results_2018

Blacklisted species:

In [5]:
blacklist = ['ribos._bound', 'ribos._free', 'tRNA_free', 'tRNA_bound', 'ATP', 'GTP', 'AMP', 'GDP', 'protein', 
             'proteins', 'peptide_bonds']

Data frame at cell level:

In [6]:
dfc = pd.DataFrame(columns=['ribosomes', 'transcripts', 'collisions', 'nocollisions', 'peptide_bonds'])

In [7]:
rows = []
dft_columns = ['length', 'init_rate', 'ribosomes', 'tic_toc', 'geneID', 'transcripts', 'index']

for i, filename in enumerate(files):
    print filename
    results = pkl.load(open(filename))
    
    # append to cell level data frame
    print 'cell level...'
    rowvals = [results['n_ribosomes'], len(results['transcriptome']), results['collisions'], 
               results['nocollisions'], results['peptide_bonds']]
    row = dict(zip(list(dfc.columns), rowvals))
    #print row
    dfc = dfc.append(row, ignore_index=True)
    #print dfc.head()
    
    # append to transcript level data frame
    print 'transcript level...'
    
    for transcript in results['transcriptome']:
        av_time = np.mean([tictoc[1] - tictoc[0] for tictoc in transcript.tic_toc])
        rowvals = [transcript.geneID, transcript.index, transcript.init_rate, transcript.length, 
                   results['n_ribosomes'], av_time, len(results['transcriptome'])]
        row = dict(zip(list(dft_columns), rowvals))
        #print row
        rows.append(row)
            
    # append to gene level data frame
    print 'gene level...'
    header = str(results['n_ribosomes']) + '_' + str(len(results['transcriptome']))
    tc = results['timecourses']
    efficiency_items = [(gene, (tc[gene][-1] - tc[gene][-6001]) / 00.) for gene in tc if gene not in blacklist]
    tmp = pd.DataFrame(efficiency_items, columns = ['gene', header]).set_index('gene')
    
    if i == 0:  # first time
        dfg = tmp
    else:
        dfg = pd.merge(dfg, tmp, left_index=True, right_index=True, how='outer')

dfc.sort_values('ribosomes', inplace=True)

dft = pd.DataFrame(rows)
dft.columns = ['ribosomes', 'transcripts', 'index', 'geneID', 'init_rate', 'length', 'tic_toc']

../../results_20180423/test configuration with 2 genes in 50 transcripts, 100 ribosomes, deltat=0.05s_20180423_1546_100_ribosomes_0200s.p
cell level...
transcript level...
gene level...


/home/martin/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/martin/.local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: list index out of range

In [ ]:
dfc

In [ ]:
dft

In [ ]:
dfg

In [ ]:
dfc.to_pickle(dirname + '/dfc' + descriptor + '.p')

In [ ]:
dft.to_pickle(dirname + '/dft' + descriptor + '.p')

In [ ]:
dfg.to_pickle(dirname + '/dfg' + descriptor + '.p')

Here we start the analysis (old notebook 17):

In [ ]:
dfc['collisions_percentage'] = dfc['collisions'] / (dfc['collisions'] + dfc['nocollisions'])

In [ ]:
dfc

In [ ]:
ax = dfc.plot(x='ribosomes', y='collisions_percentage', figsize=(10, 7), fontsize=16)
ax.legend(["5 transcripts"], prop={'size': 16})
#plt.show()

In [ ]:
dft_grouped = dft.groupby(['geneID', 'init_rate', 'length', 'ribosomes', 'transcripts'], as_index=False).mean()
dft_grouped